In [ ]:
import os
import glob
import numpy as np
import networkx as nx
import random
import math
import scipy.io
import k3d
import pickle as pkl
import matplotlib.pyplot as plt
import trimesh
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import pathlib
import json
import pickle as pkl

In [ ]:

def readOBJ(file):
	V, Vt, F, Ft = [], [], [], []
	with open(file, 'r') as f:
		T = f.readlines()
	for t in T:
		# 3D vertex
		if t.startswith('v '):
			v = [float(n) for n in t.replace('v ','').split(' ')]
			V += [v]
		# UV vertex
		elif t.startswith('vt '):
			v = [float(n) for n in t.replace('vt ','').split(' ')]
			Vt += [v]
		# Face
		elif t.startswith('f '):
			idx = [n.split('/') for n in t.replace('f ','').split(' ')]
			f = [int(n[0]) - 1 for n in idx]
			F += [f]
			# UV face
			if '/' in t:
				f = [int(n[1]) - 1 for n in idx]
				Ft += [f]
	V = np.array(V, np.float32)
	Vt = np.array(Vt, np.float32)
	F = np.array(F)
	if Ft: assert len(F) == len(Ft), 'Inconsistent .obj file, mesh and UV map do not have the same number of faces' 
	else: Vt, Ft = None, None
	return V, F, Vt, Ft

def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

def rgb_to_hex(rgb):
    if rgb.dtype == np.uint8:
        pass
    elif rgb.dtype in (np.float16, np.float32, np.float64):
        # print('Assuming Value in [0.0, 1.0]')
        rgb = (rgb * 255).astype(np.uint8)
    assert(rgb.dtype == np.uint8)
    hex = np.sum(rgb.astype(np.uint32) * np.array([1, 256, 256 ** 2])[::-1], axis=-1)
    return hex

def get_cloth_img(V, axes=(0, 2)):
	ranges = [[-1, 1], [-1, 1], [-1, 1]]
	img_shape = [256, 256]
	xy_img = np.zeros(img_shape)
			
	xy = V[:, axes,].copy()
	#center xy coordinates
	xy[:, 0] = xy[:, 0] - (np.max(xy[:, 0]) + np.min(xy[:, 0])) / 2
	xy[:, 1] = xy[:, 1] - (np.max(xy[:, 1]) + np.min(xy[:, 1])) / 2

	for i in range(len(xy)):
		canonical_coord = xy[i]
		image_coord = np.array([((canonical_coord[i] - ranges[i][0])/(ranges[i][1] - ranges[i][0]))*img_shape[i] for i in range(2)])
		xy_img[int(image_coord[0]), int(image_coord[1])] = 1
	
	return xy_img

def visualize_vertices(img, instance=None):
        plt.imshow(img)
        current_values = plt.gca().get_yticks()
        plt.gca().set_yticklabels([f'{((x-128)/128):.2f}'for x in current_values])
        plt.gca().set_xticklabels([f'{((x-128)/128):.2f}'for x in current_values])
        if instance is not None:
            plt.title(f'{instance}')
        plt.show()

def is_shirt(V, F, threshold=180):
    last_F = F.copy()[:,-1].reshape(-1, 1)
    until_last_F = F.copy()[:, :-1]

    G = nx.Graph()
    nodes = np.arange(V.shape[0])
    G.add_nodes_from(nodes)
    edges = np.array([F, np.concatenate([last_F, until_last_F], 1)]).reshape(2, -1).transpose(1, 0)
    G.add_edges_from(edges)

    N = 0
    for i in range(len(G.nodes)):
        n = len(list(nx.all_neighbors(G, i)))
        if n >= 4:
            G.remove_node(i)
            # if(n < 4):
    connected_components = list(nx.connected_components(G))

    component_lengths = []
    for c in connected_components:
        component_lengths.append(len(c))
    max_component_length = max(component_lengths)
    
    if(max_component_length < threshold):
        return True
    return False

In [ ]:


NUM_DIRS=10000

def get_all_data(input_path, save_path, num_dirs=10, mode='visualize'):
    os.chdir(input_path)
    dirs = os.listdir()
    random.shuffle(dirs)
    counter = 0
    stats = {
        'width':[],
        'height':[],
        'instance':[],
        'area':[],
    }
    print(f"{len(dirs)} directories found")
    for i, dir_idx in tqdm(zip(range(num_dirs), dirs)):

        dir_path = os.path.join(train_path, dir_idx)
        os.chdir(dir_path)
        
        # print(glob.glob("*.obj"))
        zrot = scipy.io.loadmat(glob.glob("*.mat")[0])["zrot"][0][0]
        instances = glob.glob("*.obj")
        for i, instance in enumerate(instances):

            cloth_category = instance.split('.')[0]

            full_path = os.path.join(dir_path, instance)
            V, F, Vt, Ft = readOBJ(full_path)

            rot_matrix = rotation_matrix([0, 0, 1], zrot)
            V = V @ rot_matrix

            max_height = np.max(V[:, 2])
            min_height = np.min(V[:, 2])
            height = max_height - min_height

            max_width = np.max(V[:, 0])
            min_width = np.min(V[:, 0])
            width = max_width - min_width

            # print("Height", height, "Width", width, "Depth", V[:, 1])
            max_diffs = np.max(V, axis=0) - np.min(V, axis=0)
            width = max_diffs[0]
            height = max_diffs[2]
            area = height * width
            # print(np.max(V, axis=0) - np.min(V, axis=0))
            w2h = max_diffs[0]/max_diffs[2]
            # print("width to height", w2h)
            cloth_img = get_cloth_img(V)

            if mode == 'visualize':
                visualize_vertices(cloth_img, instance)
            

            stats['height'].append(height)
            stats['width'].append(width)
            stats['instance'].append(instance)
            stats['area'].append(area)
            
            diffs = []
            mins = []
            for i in range(3):
                dim_max = np.max(V[:, i])
                dim_min = np.min(V[:, i])
                mins.append(dim_min)
                diffs.append(dim_max - dim_min)

            diffs = np.array(diffs)
            mins = np.array(mins)

            NOCS = (V.copy() - mins)/diffs

            instance_id = dir_idx + '_' + instance

            data = {
                'verts':V, 
                'nocs':NOCS,
                'faces':F,
                'height':height,
                'width':width,
                'cloth_img':cloth_img,
                'instance':instance,
                'id':instance_id,
            }

            category_path = out_path + '/' + cloth_category
            if mode == 'save':
                pathlib.Path(category_path).mkdir(exist_ok=True, parents=True)
                path = os.path.join(category_path, instance_id + '.pkl')
                with open(path, 'wb') as handle:
                    pkl.dump(data, handle)

        # df = pd.DataFrame.from_dict(stats)

train_path = "/local/crv/dataset/CLOTH3D/CLOTH3D/train/"
out_path = '/local/crv/acanberk/folding-unfolding/src/cloth_data/cloth3d_pickle'
# get_all_data(train_path, out_path, num_dirs=4000, mode='save')
 

In [ ]:
def size_filter_fn(data):
    MIN_HEIGHT = 0.6
    MAX_HEIGHT = 0.9
    height_constraint = (data['height'] < MAX_HEIGHT) and (data['height'] > MIN_HEIGHT)
    return height_constraint

def shirt_filter_fn(data):
   
    width_to_height = data['width']/data['height']
    width_to_height_constraint = width_to_height > 1.5
    
    is_shirt_not_jacket = is_shirt(data['verts'], data['faces'])

    return size_filter_fn(data) and width_to_height_constraint and is_shirt_not_jacket
    

def filter_clothes(in_path, out_path, filter_fn, train_test_split=0.7, confirm=False, save=False):

    images = []
    save_files = []

    files = os.listdir(in_path)
    for i, file in zip(range(4000), files):

        with open(os.path.join(in_path, file), 'rb') as handle:
            data = pkl.load(handle)
            if filter_fn(data):
               
                #vertical videw
                # plt.imshow(get_cloth_img(data['verts'], axes=(0, 1)))
                # plt.show()
               
                if confirm:
                    plt.imshow(data['cloth_img'])
                    plt.show()
                    r = input("Accept? (X if bad, E if escape)")
                    if r.lower() == 'e':
                        break
                    if r.lower() == 'x':
                        print("bad")
                    else:
                        print("good")
                        images.append(data['cloth_img'])
                        save_files.append(file)
                else:
                    images.append(data['cloth_img'])
                    save_files.append(file)
                    
                train_test_split_mark = int(len(save_files) * train_test_split)
                save_files_dict = {
                    'train':save_files[:train_test_split_mark],
                    'test':save_files[train_test_split_mark:]
                }
                #save json
                if save:
                    with open(os.path.join(out_path), 'w') as handle:
                        json.dump(save_files_dict, handle)
                
                print("Chosen", len(save_files), "cloth instances")

        if len(images) == 25:
            fig, axes = plt.subplots(5, 5, figsize=(10, 10))
            for i, ax in zip(range(len(images)), axes.flatten()):
                ax.imshow(images[i])
                ax.set_axis_off()
            fig.tight_layout()
            plt.show()
            images = []

shirt_path = '/local/crv/acanberk/folding-unfolding/src/cloth_data/cloth3d_pickle/Trousers'
out_path = '/local/crv/acanberk/folding-unfolding/src/cloth_data/cloth3d_pickle/pants.json'
filter_clothes(shirt_path, out_path, size_filter_fn, confirm=False, save=False)

In [ ]:
in_path = '/local/crv/acanberk/folding-unfolding/src/cloth_data/cloth3d_pickle/Tshirt'
out_path = '/local/crv/acanberk/folding-unfolding/src/cloth_data/cloth3d_pickle/longsleeve.json'

#read out path
with open(out_path, 'r') as handle:
    save_files_dict = json.load(handle)

train = save_files_dict['train']
test = save_files_dict['test']

images = []
for file in test:
    with open(os.path.join(in_path, file), 'rb') as handle:
        data = pkl.load(handle)
        images.append(data['cloth_img'])
        # plt.imshow(data['cloth_img'])
        # plt.show()
images_array = np.array(images)
plt.imshow(images_array.mean(axis=0))